In [ ]:
// Import module directly from file
import { generateMonthlySalesData, presets, seededRng, dataToCSV, analyzeTrends } from '../services/notebook_cells.module.js';

# Sales Data Generator Module Validation

This notebook validates the functionality of the sales data generator module by testing:
1. Data generation with various parameters
2. Preset configurations
3. Error handling and validation
4. Export utilities
5. Trend analysis accuracy

First, let's import the module and dependencies.

## 1. Basic Data Generation

Test the generator with default parameters:

In [ ]:
// Generate default data
const defaultData = generateMonthlySalesData();

console.log('Data points:', defaultData.length);
console.log('First record:', defaultData[0]);
console.log('Categories:', [...new Set(defaultData.map(d => d.category))]);
console.log('Date range:', defaultData[0].date, 'to', defaultData[defaultData.length - 1].date);

## 2. Parameter Validation

Test error handling for invalid inputs:

In [ ]:
// Test invalid parameters
const invalidCases = [
  { count: -1, expected: 'count must be non-negative' },
  { count: 121, expected: 'count must be ≤ 120 months' },
  { baseValue: -100, expected: 'baseValue must be between 0 and' },
  { startDate: 'invalid', expected: 'Invalid start date' },
  { categories: [], expected: 'categories array cannot be empty' },
  { seasonalityPeriod: 0, expected: 'seasonalityPeriod must be between 1 and 24' }
];

// Test each case
invalidCases.forEach(({ expected, ...params }) => {
  try {
    generateMonthlySalesData(params);
    console.error('❌ Should have thrown:', expected);
  } catch (err) {
    const passed = err.message.includes(expected);
    console.log(passed ? '✅' : '❌', 'Testing:', params, '\n   ', err.message);
  }
});

## 3. Preset Testing

Verify all presets produce valid data:

In [ ]:
// Test all presets
Object.entries(presets).forEach(([name, params]) => {
  try {
    const data = generateMonthlySalesData(params);
    console.log('✅', name, ':', 
      data.length, 'records,',
      'sales range:', Math.min(...data.map(d => d.sales)), 'to', Math.max(...data.map(d => d.sales))
    );
  } catch (err) {
    console.error('❌', name, ':', err.message);
  }
});

## 4. Seeded RNG Testing

Verify that seeded random number generation produces consistent results:

In [ ]:
// Generate two datasets with same seed
const seed = 42;
const rng1 = seededRng(seed);
const rng2 = seededRng(seed);

const data1 = generateMonthlySalesData({ rng: rng1, count: 12 });
const data2 = generateMonthlySalesData({ rng: rng2, count: 12 });

// Compare results
const matches = data1.every((d1, i) => {
  const d2 = data2[i];
  return d1.sales === d2.sales && d1.category === d2.category;
});

console.log(matches ? '✅' : '❌', 'Seeded RNG produces consistent results');
console.log('First dataset:', data1.slice(0, 3));
console.log('Second dataset:', data2.slice(0, 3));

## 5. Export Testing

Test CSV export functionality:

In [ ]:
// Test CSV export
const sampleData = generateMonthlySalesData({ count: 3 });
const csv = dataToCSV(sampleData);

console.log('CSV output:');
console.log(csv);

// Validate CSV format
const lines = csv.split('\n');
const hasHeaders = lines[0] === 'date,category,sales';
const rowCount = lines.length - 1; // -1 for header
const validFormat = lines.every(line => line.split(',').length === 3);

console.log('\nValidation:');
console.log(hasHeaders ? '✅' : '❌', 'Has correct headers');
console.log(rowCount === 3 ? '✅' : '❌', 'Has correct number of data rows');
console.log(validFormat ? '✅' : '❌', 'All rows have correct format');

## 6. Trend Analysis

Test the trend analysis functionality:

In [ ]:
// Generate data with known trend
const trendData = generateMonthlySalesData({
  count: 12,
  baseValue: 1000,
  trendPerMonth: 100,  // Should see ~100 units/month trend
  seasonalityAmplitude: 0,  // No seasonality
  noiseAmount: 0,  // No noise
  categories: ['test']  // Single category
});

const trends = analyzeTrends(trendData);
console.log('Trend analysis:', trends);

// Validate slope is close to expected
const expectedSlope = 100;
const actualSlope = trends.test.slope;
const slopeError = Math.abs(actualSlope - expectedSlope);
const maxError = 1; // Allow 1 unit of error due to rounding

console.log('\nValidation:');
console.log(slopeError <= maxError ? '✅' : '❌', 
  `Slope (${actualSlope.toFixed(2)}) matches expected (${expectedSlope})`);
console.log(trends.test.r2 > 0.99 ? '✅' : '❌',
  `R² (${trends.test.r2.toFixed(4)}) shows strong fit`);

## Next Steps

1. Run this notebook to validate core functionality
2. Run `setup-observable.ps1` to test Gist creation
3. Create an Observable notebook with the Gist URL
4. Verify imports and visualization in Observable